In [0]:
import torch as tor
import torch.nn as nn

X = tor.tensor(([1,2],[4,5],[2,9]), dtype = tor.float)
y = tor.tensor(([92],[89],[60]), dtype = tor.float)

xPredicted = tor.tensor(([4,8]),dtype = tor.float)
X_max, _ = tor.max(X,0)

xPredicted_max, _ = tor.max(xPredicted, 0)

X = tor.div(X, X_max)
xPredicted = tor.div(xPredicted, xPredicted_max)
y = y / 100 # max test score is 100

''' now comes the asli code '''

class neuralnetwork(nn.Module):
  def __init__(self, ):
    super(neuralnetwork, self).__init__()
    
    #pameters to be declared
    
    self.inputSize = 2
    self.outputSize = 1
    self.hiddenLayers = 3
    
    self.W1 = tor.randn(self.inputSize,self.hiddenLayers) #2 * 3 matrix for weights for i/p layers
    self.W2 = tor.randn(self.hiddenLayers, self.outputSize) #1* 3 matrix for weights for o/p layers
    
  def forward(self, X):
    
    self.z = tor.matmul(X, self.W1)
    self.z2 = self.sigmoid(self.z) # activation function
    self.z3 = tor.matmul(self.z2, self.W2)
    
    out = tor.sigmoid(self.z3) #activation
    
    return out
 
  def sigmoid(self, x):
    
    a = 1/(1+tor.exp(-x))
    return a
  
  def sigmoid_d1(self,x):
    
    return x * (1-x)
  
  def backward(self,X,y,o):
    
    self.out_error = y-o
    self.delta = self.sigmoid_d1(o) * self.out_error  #  h'(x) * (h(x) - y)
    self.hidd_error = tor.matmul(self.delta,tor.t(self.W2))
    self.delta2 = self.sigmoid_d1(self.hidd_error) * self.hidd_error
    
    self.W1 += tor.matmul(tor.t(X), self.delta2)
    self.W2 += tor.matmul(tor.t(self.z2), self.delta)
    
  def train(self,X,y):
    
    o = self.forward(X)
    self.backward(X,y,o)  
    
  def save_weights(self,model):
    
    tor.save(model,"NN")
    
  def predict(self):
    
    print("The predicted data based on trained weights are: ")
    print ("Input (scaled): \n" + str(xPredicted))
    print ("Output: \n" + str(self.forward(xPredicted).detach().item()*100))    


NN = neuralnetwork()
for i in range(1000):  # trains the NN 1,000 times
    #print ("#" + str(i) + " Loss: " + str(tor.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
    
NN.save_weights(NN)
NN.predict()    
  
    
    

The predicted data based on trained weights are: 
Input (scaled): 
tensor([0.5000, 1.0000])
Output: 
79.56417202949524


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type neuralnetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [29]:
import keras 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from keras import models
from keras import layers

# importing IMDB Dataset

from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)

# print(training_data.shape)  # (25000,)

data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

print("Length os data is " + str(len(data)))

'''
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))

a = 0
print(data.shape)
for i in data , a<3:
  print (i)
  a = a+1
'''
def vectorize(sequences, dimension = 10000):
 results = np.zeros((len(sequences), dimension))
 counter = 0
 for i, sequence in enumerate(sequences):
  results[i, sequence] = 1
  counter = counter + 1
 
 print("Counter is : " + str(counter))
 return results
 
data = vectorize(data)
targets = np.array(targets).astype("float32")

test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

model = models.Sequential()

# Input Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))

# Hidden Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))

# Output Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

# compiling the model
model.compile( optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
results = model.fit( train_x, train_y, epochs= 2, batch_size = 500, validation_data = (test_x, test_y))
print("Test-Accuracy:", np.mean(results.history["val_acc"]))


Length os data is 50000
Counter is : 50000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 50)                500050    
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 51        
Total params: 505,201
Trainable params: 505,201
Non-trainable params: 0
___________________________